In [42]:
import pandas as pd
import json
import shutil
import os

In [43]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [44]:
path = './results/'
normal = 'nnUNetTrainer__nnUNetPlans__3d_fullres'
da = 'nnUNetTrainerDA5__nnUNetPlans__3d_fullres'

path_axial = os.path.join(path, 'axial')
path_axial_normal = os.path.join(path_axial, normal)
path_axial_da = os.path.join(path_axial, da)


path_coronal = os.path.join(path, 'coronal')
path_coronal_normal = os.path.join(path_coronal, normal)
path_coronal_da = os.path.join(path_coronal, da)

path_sagittal = os.path.join(path, 'sagittal')
path_sagittal_normal = os.path.join(path_sagittal, normal)
path_sagittal_da = os.path.join(path_sagittal, da)

In [45]:
def create_df(view_path):
    fold = []
    patient = []
    dice = []
    TP = []
    TN = []
    FP = []
    FN = []
    n_pred = []
    n_ref = []


    for f in range(5):
        json_path = os.path.join(view_path, f'fold_{f}', 'validation', 'summary.json')

        with open(json_path) as file:
            data = json.load(file)

        for case in data['metric_per_case']:
            fold.append(f'fold_{f}')
            patient.append(case['prediction_file'].split('/')[-1])
            dice.append(case['metrics']['1']['Dice'])
            TP.append(case['metrics']['1']['TP'])
            TN.append(case['metrics']['1']['TN'])
            FP.append(case['metrics']['1']['FP'])
            FN.append(case['metrics']['1']['FN'])
            n_pred.append(case['metrics']['1']['n_pred'])
            n_ref.append(case['metrics']['1']['n_ref'])
    
    d = {
        'fold' : fold,
        'patient' : patient,
        'dice' : dice,
        'TP' : TP,
        'TN' : TN,
        'FP' : FP,
        'FN' : FN,
        'n_pred' : n_pred,
        'n_ref' : n_ref
    }

    df = pd.DataFrame(d)
    df['precision'] = df['TP'] / (df['TP'] + df['FP'])
    df['recall'] = df['TP'] / (df['TP'] + df['FN'])
    df = df.fillna(0)
    
    return df

In [46]:
axial_normal = create_df(path_axial_normal)
axial_da = create_df(path_axial_da)
axial_normal['type'] = 'normal'
axial_normal['view'] = 'axial'
axial_da['type'] = 'da'
axial_da['view'] = 'axial'


coronal_normal = create_df(path_coronal_normal)
coronal_da = create_df(path_coronal_da)
coronal_normal['type'] = 'normal'
coronal_normal['view'] = 'coronal'
coronal_da['type'] = 'da'
coronal_da['view'] = 'coronal'

sagittal_normal = create_df(path_sagittal_normal)
sagittal_da = create_df(path_sagittal_da)
sagittal_normal['type'] = 'normal'
sagittal_normal['view'] = 'sagittal'
sagittal_da['type'] = 'da'
sagittal_da['view'] = 'sagittal'

df = pd.concat([axial_normal, axial_da, coronal_normal, coronal_da, sagittal_normal, sagittal_da])
df

,fold,patient,dice,TP,TN,FP,FN,n_pred,n_ref,precision,recall,type,view
0,fold_0,1525150.nii,0.000000,0,8623817,0,183,0,183,0.000000,0.000000,normal,axial
1,fold_0,1837909.nii,0.009756,4,9877584,0,812,4,816,1.000000,0.004902,normal,axial
2,fold_0,1856568.nii,0.580206,2984,5374842,717,3601,3701,6585,0.806269,0.453151,normal,axial
3,fold_0,324035.nii,0.000000,0,4504974,231,395,231,395,0.000000,0.000000,normal,axial
4,fold_0,390232.nii,0.299465,616,5378646,772,2110,1388,2726,0.443804,0.225972,normal,axial
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,fold_4,5283984.nii,0.379966,1927,56991720,2409,3880,4336,5807,0.444419,0.331841,da,sagittal
56,fold_4,928807.nii,0.221811,360,7200954,1206,1320,1566,1680,0.229885,0.214286,da,sagittal
57,fold_4,928807_1.nii,0.000000,0,56991489,0,8447,0,8447,0.000000,0.000000,da,sagittal
58,fold_4,94628.nii,0.249403,2296,56983820,394,13426,2690,15722,0.853532,0.146037,da,sagittal


In [47]:
df.groupby(['type', 'view'])[['patient']].count()

patient
type   view             
da     axial          59
       coronal        59
       sagittal       60
normal axial          59
       coronal        59
       sagittal       60

# Dilated Dice

## Normal 

In [48]:
df_temp = df[df.type == 'normal']

axial_patients = os.listdir('../03_results_analysis/sorted/axial')

df_dilated_axial_normal = pd.read_csv('./dilated/axial_normal_dilated.csv')
df_dilated_axial_normal = df_dilated_axial_normal.fillna(0)
df_dilated_axial_normal['patient'] = axial_patients*2


df_dilated_axial_normal = df_dilated_axial_normal[['patient','Measure', 'Label_1']]

In [49]:
coronal_patients = os.listdir('../03_results_analysis/sorted/coronal')

df_dilated_coronal_normal = pd.read_csv('./dilated/coronal_normal_dilated.csv')
df_dilated_coronal_normal = df_dilated_coronal_normal.fillna(0)
df_dilated_coronal_normal = df_dilated_coronal_normal.drop(118, axis=0)
df_dilated_coronal_normal['patient'] = coronal_patients*2

df_dilated_coronal_normal = df_dilated_coronal_normal[['patient','Measure', 'Label_1']]

In [50]:
sagittal_patients = os.listdir('../03_results_analysis/sorted/sagittal')

df_dilated_sagittal_normal = pd.read_csv('./dilated/sagittal_normal_dilated.csv')
df_dilated_sagittal_normal = df_dilated_sagittal_normal.fillna(0)
df_dilated_sagittal_normal['patient'] = sagittal_patients*2

df_dilated_sagittal_normal = df_dilated_sagittal_normal[['patient','Measure', 'Label_1']]

In [51]:
df_temp['ID'] = df_temp['patient'].apply(lambda x: x.split('.')[0])

normal_axial = df_temp[df_temp.view == 'axial'].merge(df_dilated_axial_normal[df_dilated_axial_normal.Measure == 'Dilated_Dice_overlap'], left_on='ID', right_on='patient')
normal_coronal = df_temp[df_temp.view == 'coronal'].merge(df_dilated_coronal_normal[df_dilated_coronal_normal.Measure == 'Dilated_Dice_overlap'], left_on='ID', right_on='patient')
normal_sagittal = df_temp[df_temp.view == 'sagittal'].merge(df_dilated_sagittal_normal[df_dilated_sagittal_normal.Measure == 'Dilated_Dice_overlap'], left_on='ID', right_on='patient')

complete_df = pd.concat([normal_axial, normal_coronal, normal_sagittal])
complete_df = complete_df.drop(['patient_y', 'Measure', 'ID'], axis = 1)
complete_df = complete_df.rename(columns = {'Label_1':'dilated_dice'})

complete_df = complete_df.rename(columns = {'patient_x':'patient'})
complete_df_normal = complete_df.copy()
complete_df_normal

/tmp/ipykernel_32806/1084857111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['ID'] = df_temp['patient'].apply(lambda x: x.split('.')[0])


,fold,patient,dice,TP,TN,FP,FN,n_pred,n_ref,precision,recall,type,view,dilated_dice
0,fold_0,1525150.nii,0.000000,0,8623817,0,183,0,183,0.000000,0.000000,normal,axial,0.000000
1,fold_0,1837909.nii,0.009756,4,9877584,0,812,4,816,1.000000,0.004902,normal,axial,0.023171
2,fold_0,1856568.nii,0.580206,2984,5374842,717,3601,3701,6585,0.806269,0.453151,normal,axial,0.715341
3,fold_0,324035.nii,0.000000,0,4504974,231,395,231,395,0.000000,0.000000,normal,axial,0.000000
4,fold_0,390232.nii,0.299465,616,5378646,772,2110,1388,2726,0.443804,0.225972,normal,axial,0.402042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,fold_4,5283984.nii,0.389888,1658,56993089,1040,4149,2698,5807,0.614529,0.285517,normal,sagittal,0.534509
56,fold_4,928807.nii,0.097293,115,7201591,569,1565,684,1680,0.168129,0.068452,normal,sagittal,0.258460
57,fold_4,928807_1.nii,0.000000,0,56991489,0,8447,0,8447,0.000000,0.000000,normal,sagittal,0.000000
58,fold_4,94628.nii,0.116041,969,56984204,10,14753,979,15722,0.989785,0.061633,normal,sagittal,0.203700


## data aug

In [52]:
df_temp = df[df.type == 'da']

axial_patients = os.listdir('../03_results_analysis/sorted/axial')

df_dilated_axial_da = pd.read_csv('./dilated/axial_da_dilated.csv')
df_dilated_axial_da = df_dilated_axial_da.fillna(0)
df_dilated_axial_da['patient'] = axial_patients*2


df_dilated_axial_da = df_dilated_axial_da[['patient','Measure', 'Label_1']]

In [53]:
coronal_patients = os.listdir('../03_results_analysis/sorted/coronal')

df_dilated_coronal_da = pd.read_csv('./dilated/coronal_da_dilated.csv')
df_dilated_coronal_da = df_dilated_coronal_da.fillna(0)
df_dilated_coronal_da = df_dilated_coronal_da.drop(118, axis=0)
df_dilated_coronal_da['patient'] = coronal_patients*2

df_dilated_coronal_da = df_dilated_coronal_da[['patient','Measure', 'Label_1']]

In [54]:
sagittal_patients = os.listdir('../03_results_analysis/sorted/sagittal')

df_dilated_sagittal_da = pd.read_csv('./dilated/sagittal_da_dilated.csv')
df_dilated_sagittal_da = df_dilated_sagittal_da.fillna(0)
df_dilated_sagittal_da['patient'] = sagittal_patients*2

df_dilated_sagittal_da = df_dilated_sagittal_da[['patient','Measure', 'Label_1']]

In [55]:
df_temp['ID'] = df_temp['patient'].apply(lambda x: x.split('.')[0])

da_axial = df_temp[df_temp.view == 'axial'].merge(df_dilated_axial_da[df_dilated_axial_da.Measure == 'Dilated_Dice_overlap'], left_on='ID', right_on='patient')
da_coronal = df_temp[df_temp.view == 'coronal'].merge(df_dilated_coronal_da[df_dilated_coronal_da.Measure == 'Dilated_Dice_overlap'], left_on='ID', right_on='patient')
da_sagittal = df_temp[df_temp.view == 'sagittal'].merge(df_dilated_sagittal_da[df_dilated_sagittal_da.Measure == 'Dilated_Dice_overlap'], left_on='ID', right_on='patient')

complete_df = pd.concat([da_axial, da_coronal, da_sagittal])
complete_df = complete_df.drop(['patient_y', 'Measure', 'ID'], axis = 1)
complete_df = complete_df.rename(columns = {'Label_1':'dilated_dice'})

complete_df = complete_df.rename(columns = {'patient_x':'patient'})
complete_df_da = complete_df.copy()
complete_df_da

/tmp/ipykernel_32806/941643190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['ID'] = df_temp['patient'].apply(lambda x: x.split('.')[0])


,fold,patient,dice,TP,TN,FP,FN,n_pred,n_ref,precision,recall,type,view,dilated_dice
0,fold_0,1525150.nii,0.000000,0,8623726,91,183,91,183,0.000000,0.000000,da,axial,0.000000
1,fold_0,1837909.nii,0.000000,0,9877540,44,816,44,816,0.000000,0.000000,da,axial,0.000000
2,fold_0,1856568.nii,0.617212,3507,5374287,1272,3078,4779,6585,0.733836,0.532574,da,axial,0.761352
3,fold_0,324035.nii,0.000000,0,4504813,392,395,392,395,0.000000,0.000000,da,axial,0.000000
4,fold_0,390232.nii,0.129093,276,5378144,1274,2450,1550,2726,0.178065,0.101247,da,axial,0.180543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,fold_4,5283984.nii,0.379966,1927,56991720,2409,3880,4336,5807,0.444419,0.331841,da,sagittal,0.500049
56,fold_4,928807.nii,0.221811,360,7200954,1206,1320,1566,1680,0.229885,0.214286,da,sagittal,0.416513
57,fold_4,928807_1.nii,0.000000,0,56991489,0,8447,0,8447,0.000000,0.000000,da,sagittal,0.000000
58,fold_4,94628.nii,0.249403,2296,56983820,394,13426,2690,15722,0.853532,0.146037,da,sagittal,0.382142


In [56]:
df_final = pd.concat([complete_df_normal, complete_df_da])
df_final = df_final.reset_index(drop=True)

In [57]:
df_final.groupby(['view', 'type']).count()

fold  patient  dice  TP  TN  FP  FN  n_pred  n_ref  \
view     type                                                         
axial    da        59       59    59  59  59  59  59      59     59   
         normal    59       59    59  59  59  59  59      59     59   
coronal  da        59       59    59  59  59  59  59      59     59   
         normal    59       59    59  59  59  59  59      59     59   
sagittal da        60       60    60  60  60  60  60      60     60   
         normal    60       60    60  60  60  60  60      60     60   

                 precision  recall  dilated_dice  
view     type                                     
axial    da             59      59            59  
         normal         59      59            59  
coronal  da             59      59            59  
         normal         59      59            59  
sagittal da             60      60            60  
         normal         60      60            60

In [58]:
df_final.to_csv('results.csv')